This notebook showcases all the things that Cubeviz can load.

In [ ]:
import numpy as np
from astropy import units as u
from astropy.io import fits
from astropy.nddata import StdDevUncertainty
from astropy.utils.data import download_file
from astropy.wcs import WCS
from specutils import Spectrum1D

from jdaviz import Cubeviz

# FITS Data Cube (FITS File)

This file is from Cubeviz example notebook.

In [ ]:
manga_logcube = download_file('https://stsci.box.com/shared/static/28a88k1qfipo4yxc4p4d40v4axtlal8y.fits', cache=True)

In [ ]:
fits.info(manga_logcube)

In [ ]:
viz = Cubeviz()
viz.load_data(manga_logcube, data_label='manga-7495-12704-LOGCUBE')
viz.app

# FITS Data Cube (HDUList)

This is a test case adapted from Cubeviz parser test fixture, `image_hdu_obj`.

In [ ]:
flux_hdu = fits.ImageHDU(np.random.random((5, 10, 10)).astype(np.float32))
flux_hdu.name = 'FLUX'

uncert_hdu = fits.ImageHDU(np.sqrt(flux_hdu.data))
uncert_hdu.name = 'ERR'

mask_hdu = fits.ImageHDU(np.random.randint(0, 16, (5, 10, 10), dtype=np.int32))
mask_hdu.name = 'MASK'

wcs_header = {
    'WCSAXES': 3, 'CRPIX1': 38.0, 'CRPIX2': 38.0, 'CRPIX3': 1.0,
    'PC1_1 ': -0.000138889, 'PC2_2 ': 0.000138889,
    'PC3_3 ': 8.33903304339E-11, 'CDELT1': 1.0, 'CDELT2': 1.0,
    'CDELT3': 1.0, 'CUNIT1': 'deg', 'CUNIT2': 'deg', 'CUNIT3': 'm',
    'CTYPE1': 'RA---TAN', 'CTYPE2': 'DEC--TAN', 'CTYPE3': 'WAVE-LOG',
    'CRVAL1': 205.4384, 'CRVAL2': 27.004754, 'CRVAL3': 3.62159598486E-07,
    'LONPOLE': 180.0, 'LATPOLE': 27.004754, 'MJDREFI': 0.0,
    'MJDREFF': 0.0, 'DATE-OBS': '2014-03-30',
    'RADESYS': 'FK5', 'EQUINOX': 2000.0}

flux_hdu.header.update(wcs_header)
flux_hdu.header['BUNIT'] = '1E-17 erg*s^-1*cm^-2*Angstrom^-1*pix^-1'

hdu_list = fits.HDUList([fits.PrimaryHDU(), flux_hdu, uncert_hdu, mask_hdu])
hdu_list.info()

In [ ]:
viz = Cubeviz()
viz.load_data(hdu_list, data_label='mycube')
viz.app

# FITS Data Cube (Individual HDUs)

This reuses HDUs created above. Instead of passing in HDUList, we load one HDU at a time.

In [ ]:
viz = Cubeviz()
viz.load_data(flux_hdu, data_label='myhdu[left]')
viz.load_data(uncert_hdu, data_label='myhdu[center]')
viz.load_data(mask_hdu, data_label='myhdu[right]')
viz.app

# HST WFC3/IR Ramp (Spectrum1D, 3D)

This is a little more complicated because we need to stack all the different 2D extensions into respective cubes. We do not attempt to recreate a composite WCS here, but rather just use a dummy one.

In [ ]:
wfc3ir_file = download_file('https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:HST/product/icgk01a8q_ima.fits', cache=True)
fits.info(wfc3ir_file)

In [ ]:
sci_unit = u.electron / u.s  # From BUNIT but its value is invalid for astropy.units
err_unit = sci_unit  # Assume the same

with fits.open(wfc3ir_file) as pf:
    sci_cube = np.stack([hdu.data for hdu in pf if hdu.name == 'SCI']) * sci_unit
    err_cube = np.stack([hdu.data for hdu in pf if hdu.name == 'ERR']) * err_unit
    dq_cube = np.stack([hdu.data for hdu in pf if hdu.name == 'DQ']) * u.dimensionless_unscaled
    
uncertainty = StdDevUncertainty()
uncertainty.array = err_cube

In [ ]:
# Assign dummy WCS for now. Constructing actual WCS is out of scope here.
from jdaviz.configs.cubeviz.plugins.parsers import generate_dummy_fits_wcs_3d
fake_wcs = generate_dummy_fits_wcs_3d()

In [ ]:
sc = Spectrum1D(flux=sci_cube, uncertainty=uncertainty, mask=dq_cube, wcs=fake_wcs)

In [ ]:
viz = Cubeviz()
viz.load_data(sc, data_label='icgk01a8q_ima.fits')
viz.app

# JWST s3d (FITS File)

In [ ]:
# Ask Brian Cherinka if you want the file.
s3d_filename = 'jw00619-o094_t001_miri_ch1-long_s3d.fits'

In [ ]:
viz = Cubeviz()
viz.load_data(s3d_filename)
viz.app

# Spectrum1D (1D)

This is from `specutils` example.

In [ ]:
sp_filename = download_file('https://data.sdss.org/sas/dr16/sdss/spectro/redux/26/spectra/1323/spec-1323-52797-0012.fits', cache=True)
fits.info(sp_filename)

In [ ]:
with fits.open(sp_filename) as pf:
    lamb = 10 ** pf[1].data['loglam'] * u.AA 
    flux = pf[1].data['flux'] * 10**-17 * u.Unit('erg cm-2 s-1 AA-1')
    sp = Spectrum1D(spectral_axis=lamb, flux=flux)

In [ ]:
viz = Cubeviz()
viz.load_data(sp, data_label='spec-1323-52797-0012')
viz.app

# Numpy Array (3D)

In [ ]:
sci_arr = np.random.random((5, 10, 10))
err_arr = np.sqrt(sci_arr)

In [ ]:
viz = Cubeviz()
viz.load_data(sci_arr, data_type='flux', data_label='myarray[SCI]')
viz.load_data(err_arr, data_type='uncert', data_label='myarray[ERR]')
viz.app

# Invalid Data

Cubeviz now should throw exception instead of giving misleading success message.

In [ ]:
image2d = np.random.random((10, 10))

In [ ]:
viz = Cubeviz()
viz.load_data(image2d)